In [2]:
import os
os.chdir('..')
%pwd

'c:\\Users\\smaji\\Music\\MLFLOW\\datascienceproject'

In [18]:
from dataclasses import dataclass

@dataclass
class ModelTrainerConfig:
    root_dir: str
    train_data_path: str
    test_data_path: str
    model_name: str
    alpha: float 
    l1_ratio: float 
    target_column: str 


In [19]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories([self.config.artifacts_root])
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column=schema.name
        )

        return model_trainer_config

In [21]:
import pandas as pd
import os 
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        os.makedirs(self.config.root_dir, exist_ok=True)  # Ensure folder exists
        joblib.dump(lr, model_path)
        logger.info(f"Model saved at {model_path}")



    

In [22]:
model_path = "artifacts/model_trainer/model.joblib"
if os.path.exists(model_path):
    model = joblib.load(model_path)
else:
    print("Model file not found. Please run training first.")

Model file not found. Please run training first.


In [23]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-08-29 21:59:13,458: - INFO: -common: - yaml file: config\config.yaml loaded successfully]
[2025-08-29 21:59:13,461: - INFO: -common: - yaml file: params.yaml loaded successfully]
[2025-08-29 21:59:13,463: - INFO: -common: - yaml file: schema.yaml loaded successfully]
[2025-08-29 21:59:13,465: - INFO: -common: - created directory at: artifacts]
[2025-08-29 21:59:13,466: - INFO: -common: - created directory at: artifacts]
[2025-08-29 21:59:13,522: - INFO: -2390489397: - Model saved at artifacts/model_trainer\model.joblib]
